In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow import keras

Using TensorFlow backend.


In [2]:
# Specify model for data handling
# design network
n_hidden1 = 10
activation1 = 'relu'
activation2 = 'relu'
n_hidden2 = 10
EPOCHS = 100
BATCH_SIZE = 100

rnn_flag = False

In [3]:
# Create directory for Tensorboard
import os, time, datetime
stamp = datetime.datetime.now()
stamp = time.mktime(stamp.timetuple())
cwd = os.getcwd()
PATH_TO_LOG = cwd+'\\tfboards\\run_'+str(stamp) + '_' + activation1 +'x'+activation2 + '-' +str(n_hidden1) + 'x' + str(n_hidden2)
os.mkdir(PATH_TO_LOG)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=PATH_TO_LOG)

In [4]:
# Handle data.
df = pd.read_csv('../data/20190606180000-20190606190000.csv')
df=df.replace([np.inf, -np.inf], np.nan)
df=df.dropna(axis=0);


# columns of interest
cols = ['B:VIMIN', 'B_VIMIN','B:IMINER']

# Extract data we're focused on
df2=df[cols]
#df2=df2.set_index(pd.DatetimeIndex(df2['time'])) # set index to time (not in-place operation)

# clean data: remove rows with spurious values of B:VIMIN (>= 3 stdevs)
std_vimin  = np.std(df['B:VIMIN'])
mean_vimin = np.mean(df['B:VIMIN'])
df2=df2[df2['B:VIMIN']< mean_vimin + 3*std_vimin]
print(df2.shape)

C:\Users\Rachael\.conda\envs\py3tf1gnt\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (77,79,81,83,85,87) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(48004, 3)


In [5]:
# FUNCTION: series_to_supervised
# INPUT: data   ~ numpy array of time series observations
#       n_lag   ~ number of lag observations as input 
#       n_out   ~ number of observations as output.
#       dropnan ~ Boolean whether or not to drop rows with NaN values.
# sourced: https://machinelearningmastery.com/multivariate-time-series-forecasting-lstms-keras/
def series_to_supervised(data, n_lag=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = pd.DataFrame(data)
    col_data = []
    col_names = []
    # build the input sequence by concatenating farthest to closest in time
    for i in range(n_lag, 0, -1):
        col_data.append(df.shift(i))
        col_names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # build the output sequence
    for i in range(0, n_out):
        col_data.append(df.shift(-i))
        if i == 0:
            col_names += [('x%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            col_names += [('x%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    agg = pd.concat(col_data, axis=1)
    agg.columns =col_names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [6]:
# select B:VIMIN to convert to lagged time series
cycle  = 15
scaler = MinMaxScaler(feature_range=(0, 1))
df_xy  = pd.DataFrame(scaler.fit_transform(df2.values))
# convert B:VIMIN to lagged series
df_ts = pd.DataFrame(series_to_supervised(list(df_xy.iloc[:,0]),cycle))
df_ts=df_ts.reset_index()

# join lagged time series with the non-lagged features AND target: B_VIMIN (a feature) and target 
data = pd.concat([df_ts, df_xy.iloc[:, 1:]], axis = 1,ignore_index=True)

In [7]:
data= data.drop(data.columns[0],axis=1)

In [8]:
data=data.dropna(axis=0)

In [ ]:
p = 0.8 # percent of data to take
n_train = int(len(data)* p)


train = data.sample(n= n_train, random_state = 42)
test = data.drop(train.index)


# split into input and outputs
x_train, y_train = train.iloc[:, :-1], train.iloc[:, -1]
x_test, y_test = test.iloc[:, :-1], test.iloc[:, -1]

#L RNN LSTM Model only
if rnn_flag == True:
# reshape input to be 3D [samples, timesteps, features]
    x_train = x_train.reshape((x_train.shape[0], 1, x_train.shape[1]))
    x_test = x_test.reshape((x_test.shape[0], 1, x_test.shape[1]))
print(x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(38391, 17) (38391,) (9598, 17) (9598,)


In [ ]:
# Construct Newtork
model = Sequential()

model.add(Dense(n_hidden1, activation=activation1, input_dim=(x_train.shape[1]))) # input to hidden1
model.add(Dense(n_hidden2, activation=activation2)) # hidden1 to hidden2
model.add(Dense(1, activation='linear')) # output layer


model.compile(optimizer='rmsprop',
              loss='mae',
              metrics=['mae'])

# fit network
history = model.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_data=(x_test, y_test), verbose=2, shuffle=False, callbacks=[tensorboard_callback])

W0806 20:49:57.325581 15452 deprecation_wrapper.py:118] From C:\Users\Rachael\.conda\envs\py3tf1gnt\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 20:49:57.351126 15452 deprecation_wrapper.py:118] From C:\Users\Rachael\.conda\envs\py3tf1gnt\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 20:49:57.359193 15452 deprecation_wrapper.py:118] From C:\Users\Rachael\.conda\envs\py3tf1gnt\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 20:49:57.411992 15452 deprecation_wrapper.py:118] From C:\Users\Rachael\.conda\envs\py3tf1gnt\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0806 20:49:57.567309 15

Train on 38391 samples, validate on 9598 samples
Epoch 1/100
 - 2s - loss: 0.1090 - mean_absolute_error: 0.1090 - val_loss: 0.0685 - val_mean_absolute_error: 0.0685
Epoch 2/100
 - 1s - loss: 0.0653 - mean_absolute_error: 0.0653 - val_loss: 0.0608 - val_mean_absolute_error: 0.0608
Epoch 3/100
 - 1s - loss: 0.0596 - mean_absolute_error: 0.0596 - val_loss: 0.0564 - val_mean_absolute_error: 0.0564
Epoch 4/100
 - 0s - loss: 0.0563 - mean_absolute_error: 0.0563 - val_loss: 0.0563 - val_mean_absolute_error: 0.0563
Epoch 5/100
 - 1s - loss: 0.0546 - mean_absolute_error: 0.0546 - val_loss: 0.0547 - val_mean_absolute_error: 0.0547
Epoch 6/100
 - 2s - loss: 0.0537 - mean_absolute_error: 0.0537 - val_loss: 0.0521 - val_mean_absolute_error: 0.0521
Epoch 7/100
 - 2s - loss: 0.0531 - mean_absolute_error: 0.0531 - val_loss: 0.0520 - val_mean_absolute_error: 0.0520
Epoch 8/100
 - 2s - loss: 0.0526 - mean_absolute_error: 0.0526 - val_loss: 0.0519 - val_mean_absolute_error: 0.0519
Epoch 9/100
 - 2s - los